# WDDL 

In [1]:
import os
import numpy as np
import pandas as pd
import datetime
import argparse
import matplotlib.pyplot as plt
from scipy import signal as signal

In [2]:
def frread(fname=None):
    """
    This is a rough translation of frread.m from J. Lichtenberger for the
    STEREO=TRUE case, i.e. we assume orthogonal loop antenna.

    INPUTS
        fname (string)	File name path to the .vr2 file to load

    OUTPUTS
        wh (ndarray)	2xN array with the two traces in the first and second rows.
    """

    # open file for reading
    fid = open(fname, 'rb')

    # get data from file - 16-bit signed integers
    dat = np.fromfile(fid, dtype=np.int16)

    # length of one frame
    frLen = 4103  ## not sure how this is determined

    # number of frames to read
    nFrameRead = len(dat) / frLen

    # data length of frame
    adatlen = 2048

    # length of data set
    N = int(nFrameRead * adatlen)
    wh = np.zeros((N, 2), dtype=float)

    # for every frame
    for i in np.arange(0, nFrameRead, dtype=int):
        # indices for first component
        i1 = np.arange(7 + i * frLen, (i + 1) * frLen, 2, dtype=int)
        # indices for second component
        i2 = np.arange(8 + i * frLen, (i + 1) * frLen + 0, 2, dtype=int)
        ii = np.arange(i * adatlen, (i + 1) * adatlen, dtype=int)
        wh[ii, 0] = dat[i1]
        wh[ii, 1] = dat[i2]

    print(len(np.arange(0, nFrameRead, dtype=int)))
    return wh

In [3]:
def extract_data(data_dir, data_site,fname):
    # read vr2 file
    wh = frread(os.path.join(data_dir,data_site,fname)) # fix this join
    
    # CONSTANTS
    # Sampling frequency (20kHz for SANAE, 40kHz for MARION )
    fs = 2e4 if data_site=="sanae" else 4e4
    # time step in microseconds (for dataframe index)
    dt = 1e6 / fs

    # Set the date/time format in the filename
    # dtFormat = '%Y-%m-%dUT%H_%M_%S.%f'
    dtFormat = '%Y-%m-%dUT%H_%M_%S.%f'

    # Set up pandas dataframe
    # Start time
    t0 = pd.datetime.strptime(fname[0:27], dtFormat)
    # Number of samples
    Nsamples = len(wh[:, 0])
    # End time
    t1 = t0 + datetime.timedelta(0, 0, Nsamples * dt)
    # Create index
    tindex = pd.date_range(start=t0, periods=Nsamples, freq='50U') # freq = 50us

    # Create pandas data frame from wh
    whdf = pd.DataFrame(index=tindex, data=wh[:, 0], columns=['X'])
    whdf['Y'] = wh[:, 1]
    # The 'X' and 'Y' columns are the signal received by the North/South and
    # East/West pointing orthogonal loop antennas used at Marion and SANAE
    
    t0str = pd.datetime.strftime(t0, '%Y-%m-%d %H:%M:%S.%f')
    t1str = pd.datetime.strftime(t1, '%Y-%m-%d %H:%M:%S.%f')
    
    return whdf, fs, t0str, t1str

In [4]:
def spectrogram(data,sample, t0, t1):
#     # Make a nice string
    print('%s -- %s' % (t0str, t1str))

    fig = plt.figure()
    ax1 = plt.gca()
    s, f, t, img = ax1.specgram(data.X.values, Fs=sample, detrend='linear', NFFT=512 , noverlap=64, scale='dB',
                                scale_by_freq=False, cmap='jet')
    fig.colorbar(mappable=img, label='Power [dB]')
    fig.savefig(os.path.join(os.getcwd(), 'image.jpeg'))
    plt.show()
    
    return s,f,t,img

In [9]:
# Data parameters
data_root = '.'
awd_event = 0
site = 'marion'

# load all data file
data_location = os.path.join(data_root,'awdEvents'+str(awd_event),site)
data_capture = os.listdir(os.path.join(data_root,'awdEvents'+str(awd_event),site,site+'_data'))
print(data_capture)
data, sample, t0, t1 = extract_data(data_location, site+'_data', '2013-01-19UT04:50:31.88387602.marion.vr2')
spectrum,frequencies,times, image = spectrogram(data, sample, t0, t1)

['2013-01-03UT00:21:40.65827602.marion.vr2', '2013-01-05UT20:29:42.19427602.marion.vr2', '2013-01-05UT20:57:25.73347602.marion.vr2', '2013-01-03UT04:52:57.18947602.marion.vr2', '2013-01-02UT20:28:41.36867602.marion.vr2', '2013-01-05UT20:16:13.33667594.marion.vr2', '.ipynb_checkpoints', 'Data_rename.ipynb', '2013-01-02UT23:26:15.01347602.marion.vr2', '2013-01-05UT19:20:27.92867594.marion.vr2', '2013-01-05UT19:55:16.27427594.marion.vr2', '2013-01-05UT19:20:28.03107594.marion.vr2', '2013-12-25UT00:04:36.38579414.marion.vr2', '2013-01-05UT19:13:34.64227602.marion.vr2']


FileNotFoundError: [Errno 2] No such file or directory: './awdEvents0/marion/marion_data/2013-01-19UT04:50:31.88387602.marion.vr2'